# Create a nanoribbon material with the digits (or letters) on a nanoribbon.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.made.tools.build.nanoribbon import create_nanoribbon, NanoribbonConfiguration
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.modify import rotate
from IPython.display import display
import numpy as np


DIGITS_STRING = "2026"
grid_size = 10

SHOW_DIGITS = False

combined_length = 32
material = Material(Material.default_config)
graphene = Material(Materials.get_by_name_first_match("Graphene"))
config = NanoribbonConfiguration(material=graphene, width=4 * 10, length=combined_length, edge_type="zigzag")
nanoribbon = create_nanoribbon(config)


In [ ]:
def generate_digit_grid(digit, grid_size=10):
    """
    Generate a binary grid for a digit and return ON pixel coordinates.
    """
    from PIL import Image, ImageDraw, ImageFont
    
    image = Image.new("1", (grid_size, grid_size), color=1)
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.load_default()
    except IOError:
        raise RuntimeError("Failed to load bitmap font.")
    
    draw.text((0, 0), digit, fill=0, font=font)
    grid = np.array(image)
    
    on_pixels = [(x, y) for y in range(grid.shape[0]) for x in range(grid.shape[1]) if not grid[y, x]]
    return on_pixels

def visualize_digit_coords(coords, grid_size=10):
    """
    Visualize digit coordinates as a grid and image.
    """
    from PIL import Image
    
    grid = np.ones((grid_size, grid_size), dtype=int)
    for x, y in coords:
        if 0 <= y < grid_size and 0 <= x < grid_size:
            grid[y, x] = 0
    
    print("   ", end="")
    for x in range(grid_size):
        print(f"{x:2}", end=" ")
    print()
    
    for y in range(grid_size):
        print(f"{y:2} ", end="")
        for x in range(grid_size):
            print(f"{grid[y, x]:2}", end=" ")
        print()
    
    image = Image.fromarray((grid * 255).astype(np.uint8), mode='L')
    scale_factor = 10
    scaled_image = image.resize((grid_size * scale_factor, grid_size * scale_factor), resample=Image.NEAREST)
    display(scaled_image)
    
    print(f"\nON pixels: {coords}")

def toggle_pixels(coords, toggles):
    """
    Toggle pixels in the coordinate list.
    If a coordinate is in the list, remove it. If not, add it.
    """
    coords_set = set(coords)
    for toggle in toggles:
        if toggle in coords_set:
            coords_set.remove(toggle)
        else:
            coords_set.add(toggle)
    return list(coords_set)


In [ ]:

digits_coords = []
for i, digit in enumerate(DIGITS_STRING):
    coords = generate_digit_grid(digit, grid_size)
    digits_coords.append(coords)
    if SHOW_DIGITS:
        print(f"Digit {i} ('{digit}'):")
        visualize_digit_coords(coords, grid_size)
        print()


In [ ]:
adjustments = {
    # 3: [(5, 4)],
}


for digit_index, toggles in adjustments.items():
    digits_coords[digit_index] = toggle_pixels(digits_coords[digit_index], toggles)
    print(f"Adjusted Digit {digit_index} ('{DIGITS_STRING[digit_index]}'):")
    visualize_digit_coords(digits_coords[digit_index], grid_size)
    print()


In [ ]:
def create_material_from_digit_coords(digit_coords_list, nanoribbon, grid_size, lattice_param=2.46, spacing=0, x_shift=-5):
    """
    Create material with atoms positioned based on digit coordinates.
    """
    material = nanoribbon.clone()
    x_offset = 0
    
    for digit_coords in digit_coords_list:
        x_offset += grid_size * lattice_param + spacing * lattice_param + x_shift
        
        for x, y in digit_coords:
            coord = np.array([
                x * lattice_param + x_offset - lattice_param * 7,
                y * lattice_param + lattice_param * 2 + 90,
                2 * lattice_param
            ])
            material.add_atom("Au", coord, use_cartesian_coordinates=True)
            
            mirror_coord = np.array([
                -coord[0],
                coord[1],
                5 * lattice_param
            ])
            material.add_atom("Cs", mirror_coord, use_cartesian_coordinates=True)
    
    return material


In [ ]:
lattice_param = 2.46
spacing = 0
x_shift = -5

material = create_material_from_digit_coords(digits_coords, nanoribbon, grid_size, lattice_param, spacing, x_shift)


In [ ]:
from utils.jupyterlite import set_materials
from utils.visualize import visualize_materials
from mat3ra.made.tools.modify import add_vacuum, translate_to_z_level

material_copy = material.clone()
material_copy = add_vacuum(material_copy, 20, to_bottom=True)
material_copy = translate_to_z_level(material_copy, "center")
material_copy = rotate(material_copy, [1, 0, 0], -90, rotate_cell=False)

visualize_materials([material_copy], rotation="-180x")
visualize_materials([material_copy], rotation="-90x")

material_copy.name = DIGITS_STRING
set_materials([material_copy])
